In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1570414361246_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
taxi_data = glueContext.create_dynamic_frame.from_catalog(database="reinvent19", table_name="canonical")
print("2018/2019 Taxi Data Count: ", taxi_data.count())
taxi_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2018/2019 Taxi Data Count:  452091107
root
|-- pickup_datetime: timestamp
|-- dropoff_datetime: timestamp
|-- pulocationid: long
|-- dolocationid: long
|-- type: string
|-- vendorid: string

The records in your input files should contain the following fields:

    start—A string with the format YYYY-MM-DD HH:MM:SS. The start timestamp can't contain time zone information.

    target—An array of floating-point values or integers that represent the time series. You can encode missing values as null literals, or as "NaN" strings in JSON, or as nan floating-point values in Parquet.

    dynamic_feat (optional)—An array of arrays of floating-point values or integers that represents the vector of custom feature time series (dynamic features). If you set this field, all records must have the same number of inner arrays (the same number of feature time series). In addition, each inner array must have the same length as the associated target value. Missing values are not supported in the features. For example, if target time series represents the demand of different products, an associated dynamic_feat might be a boolean time-series which indicates whether a promotion was applied (1) to the particular product or not (0):

    {"start": ..., "target": [1, 5, 10, 2], "dynamic_feat": [[0, 1, 1, 0]]}

    cat (optional)—An array of categorical features that can be used to encode the groups that the record belongs to. Categorical features must be encoded as a 0-based sequence of positive integers. For example, the categorical domain {R, G, B} can be encoded as {0, 1, 2}. All values from each categorical domain must be represented in the training dataset. That's because the DeepAR algorithm can forecast only for categories that have been observed during training. And, each categorical feature is embedded in a low-dimensional space whose dimensionality is controlled by the embedding_dimension hyperparameter. For more information, see DeepAR Hyperparameters.

If you use a JSON file, it must be in JSON Lines format. For example:

{"start": "2009-11-01 00:00:00", "target": [4.3, "NaN", 5.1, ...], "cat": [0, 1], "dynamic_feat": [[1.1, 1.2, 0.5, ...]]}
{"start": "2012-01-30 00:00:00", "target": [1.0, -5.0, ...], "cat": [2, 3], "dynamic_feat": [[1.1, 2.05, ...]]}
{"start": "1999-01-30 00:00:00", "target": [2.0, 1.0], "cat": [1, 4], "dynamic_feat": [[1.3, 0.4]]}

In this example, each time series has two associated categorical features and one time series features. 

In [3]:
taxi_data.toDF().show(30, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+------------+------------+----+--------+
|pickup_datetime    |dropoff_datetime|pulocationid|dolocationid|type|vendorid|
+-------------------+----------------+------------+------------+----+--------+
|2018-01-30 21:15:34|null            |129         |null        |fhv |fhv     |
|2018-01-30 21:35:29|null            |112         |null        |fhv |fhv     |
|2018-01-30 21:16:34|null            |42          |null        |fhv |fhv     |
|2018-01-30 21:40:35|null            |131         |null        |fhv |fhv     |
|2018-01-30 21:49:59|null            |121         |null        |fhv |fhv     |
|2018-01-30 21:44:55|null            |235         |null        |fhv |fhv     |
|2018-01-30 21:51:30|null            |235         |null        |fhv |fhv     |
|2018-01-30 22:15:28|null            |208         |null        |fhv |fhv     |
|2018-01-30 21:46:49|null            |265         |null        |fhv |fhv     |
|2018-01-30 21:35:23|null            |29          |n

We need to restructure this so that each time is a single row, and the time series values are in the series, followed by the numerical and categorical features


## first let's re-set the interval to a finer grain,  right now looking at things per second is too fine grain.

In [4]:
df = taxi_data.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.functions import col, max as max_, min as min_

day = 60 * 60 * 24
epoch = (col("pickup_datetime").cast("bigint") / day).cast("bigint") * day

with_epoch = df.withColumn("epoch", epoch)

min_epoch, max_epoch = with_epoch.select(min_("epoch"), max_("epoch")).first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Reference range 
ref = spark.range(
    min_epoch, max_epoch + 1, day
).toDF("epoch")

resampled_df = (ref
    .join(with_epoch, "epoch", "left")
    .orderBy("epoch")
    .withColumn("ts_resampled", col("epoch").cast("timestamp")))

resampled_df.cache()

resampled_df.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------------+------------+------------+------+--------+-------------------+
|epoch    |pickup_datetime    |dropoff_datetime   |pulocationid|dolocationid|type  |vendorid|ts_resampled       |
+---------+-------------------+-------------------+------------+------------+------+--------+-------------------+
|978307200|2001-01-01 00:09:39|2001-01-01 06:39:54|48          |148         |yellow|2       |2001-01-01 00:00:00|
|978307200|2001-01-01 00:02:08|2001-01-01 01:00:02|151         |151         |yellow|2       |2001-01-01 00:00:00|
|978307200|2001-01-01 00:02:26|2001-01-01 00:04:49|48          |48          |yellow|2       |2001-01-01 00:00:00|
|978307200|2001-01-01 22:55:43|2001-01-02 00:05:43|234         |162         |yellow|2       |2001-01-01 00:00:00|
|978307200|2001-01-01 00:01:48|2001-01-01 00:15:47|43          |170         |yellow|2       |2001-01-01 00:00:00|
|978307200|2001-01-01 00:05:12|2001-01-01 00:09:59|140         |229         |yellow|2   

You can see now that we are resampling per day the resample column, in which we can now aggregate across.

In [7]:
from pyspark.sql import functions as func
count_per_day_resamples = resampled_df.groupBy(["ts_resampled", "type", "pulocationid", "dolocationid"]).count()
count_per_day_resamples.cache()
count_per_day_resamples.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+------------+------------+-----+
|ts_resampled       |type  |pulocationid|dolocationid|count|
+-------------------+------+------------+------------+-----+
|2018-01-07 00:00:00|yellow|162         |142         |127  |
|2018-01-07 00:00:00|yellow|144         |249         |117  |
|2018-01-07 00:00:00|yellow|138         |7           |107  |
|2018-01-07 00:00:00|yellow|170         |90          |126  |
|2018-01-07 00:00:00|yellow|90          |231         |89   |
|2018-01-07 00:00:00|yellow|161         |75          |49   |
|2018-01-07 00:00:00|yellow|114         |113         |143  |
|2018-01-07 00:00:00|yellow|114         |209         |19   |
|2018-01-07 00:00:00|yellow|151         |74          |59   |
|2018-01-07 00:00:00|yellow|186         |68          |201  |
+-------------------+------+------------+------------+-----+
only showing top 10 rows

## TODO -- Right now the "null" column is showing up instead of the fhvhv.  

In [25]:
time_series = count_per_day_resamples.groupBy(["ts_resampled", "pulocationid", "dolocationid"])\
.pivot('type').sum("count").drop("null").cache()
time_series.show(30,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+------------+----+-----+------+
|ts_resampled       |pulocationid|dolocationid|fhv |green|yellow|
+-------------------+------------+------------+----+-----+------+
|2019-05-27 00:00:00|215         |262         |null|null |3     |
|2019-05-29 00:00:00|140         |174         |null|null |1     |
|2019-05-29 00:00:00|119         |254         |null|1    |null  |
|2019-06-02 00:00:00|220         |241         |null|1    |null  |
|2019-05-30 00:00:00|244         |24          |null|7    |3     |
|2019-06-04 00:00:00|250         |250         |null|1    |null  |
|2019-06-05 00:00:00|246         |112         |null|null |6     |
|2019-06-07 00:00:00|145         |137         |null|null |2     |
|2019-06-09 00:00:00|236         |100         |null|null |29    |
|2019-06-14 00:00:00|98          |264         |null|null |2     |
|2019-06-16 00:00:00|66          |138         |null|1    |1     |
|2019-06-10 00:00:00|229         |185         |null|null |1     |
|2018-01-2

In [26]:
time_series.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8755220

## now that we have 8 million entries -- let's send this back to the local python

In [27]:
%%spark -o time_series

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## we are in the local panda/python environment now


In [29]:
%%local
time_series.dtypes

ts_resampled    datetime64[ns]
pulocationid             int64
dolocationid           float64
fhv                    float64
green                  float64
yellow                 float64
dtype: object